In [1]:
!pip install pandas numpy scikit-learn xgboost lightgbm catboost matplotlib seaborn openpyxl joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.3 MB/s eta 0:00:00


##English

In [2]:
import pandas as pd
import numpy as np
import warnings
import time
import os
from collections import defaultdict

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, roc_curve)

# Models
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, VotingClassifier, StackingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Advanced models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("Warning: XGBoost not available")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("Warning: LightGBM not available")

try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("Warning: CatBoost not available")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# CONFIGURATION
# ============================================================================

# FILE PATHS - UPDATE THESE WITH YOUR ACTUAL FILE PATHS
FILE_MAPPING = {
    'simple': {
        'ai': 'AI_english_pmi_graph_analysis_sp_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_sp_nolemma.xlsx'
    },
    'simple_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_sp_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_sp_lemma.xlsx'
    },
    'nltk': {
        'ai': 'AI_english_pmi_graph_analysis_adv_nltk_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_nltk_nolemma.xlsx'
    },
    'nltk_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_adv_nltk_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_nltk_lemma.xlsx'
    },
    'spacy': {
        'ai': 'AI_english_pmi_graph_analysis_adv_spacy_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_spacy_nolemma.xlsx'
    },
    'spacy_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_adv_spacy_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_spacy_lemma.xlsx'
    }
}

# Statistical test results file
BEST_FEATURES_FILE = 'best_features_summary.xlsx'

# Output directory
OUTPUT_DIR = 'ml_classification_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Parameters
TEST_SIZE = 0.2
RANDOM_SEED = 42
CV_FOLDS = 5
TOP_N_MODELS_TO_TUNE = 3

# Class labels (documented clearly)
# 0 = AI-generated text
# 1 = Human-written text
AI_LABEL = 0
HUMAN_LABEL = 1

print("=" * 80)
print("AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM")
print("=" * 80)
print(f"Class Labels: AI = {AI_LABEL}, Human = {HUMAN_LABEL}")
print(f"Test Size: {TEST_SIZE}, CV Folds: {CV_FOLDS}, Random Seed: {RANDOM_SEED}")
print("=" * 80 + "\n")

# ============================================================================
# SOURCE NAME MAPPING
# ============================================================================

def map_source_name(source_str):
    """Map source names from Excel to file mapping keys"""
    mapping = {
        'Spacy Token': 'spacy',
        'Spacy Lemma': 'spacy_lemma',
        'Simple Token': 'simple',
        'Simple Lemma': 'simple_lemma',
        'Nltk Token': 'nltk',
        'Nltk Lemma': 'nltk_lemma'
    }
    return mapping.get(source_str, source_str.lower().replace(' ', '_'))

# ============================================================================
# DATA LOADING FUNCTIONS
# ============================================================================

def load_feature_recommendations(filepath):
    """Load and filter highly recommended features with their sources"""
    print(f"Loading feature recommendations from: {filepath}")

    df = pd.read_excel(filepath)

    # Filter only highly recommended features
    highly_recommended = df[df['feature_recommendation'] == 'Highly Recommended'].copy()

    print(f"Total features in file: {len(df)}")
    print(f"Highly recommended features: {len(highly_recommended)}")

    # Create feature-source mapping
    feature_source_map = {}
    for _, row in highly_recommended.iterrows():
        feature_name = row['metric']
        source = map_source_name(row['source'])
        feature_source_map[feature_name] = source

    print(f"\nHighly Recommended Features and Their Sources:")
    for feature, source in feature_source_map.items():
        print(f"  - {feature}: {source}")

    return feature_source_map

def load_excel_data(filepath, max_rows=300):
    """Load Excel file and clean it"""
    df = pd.read_excel(filepath)

    # Remove 'Average' row if exists
    df = df[df['Graph'] != 'Average'].copy()

    # Remove 'Graph' column
    if 'Graph' in df.columns:
        df = df.drop('Graph', axis=1)

    # Keep only first max_rows
    df = df.head(max_rows)

    return df

def build_mixed_source_dataset(feature_source_map, file_mapping):
    """
    Build dataset by selecting each feature from its best-performing source
    """
    print("\n" + "=" * 80)
    print("BUILDING MIXED-SOURCE DATASET")
    print("=" * 80)

    ai_features = []
    human_features = []

    for feature_name, source in feature_source_map.items():
        print(f"\nProcessing feature: {feature_name} from source: {source}")

        try:
            # Load AI data
            ai_file = file_mapping[source]['ai']
            ai_df = load_excel_data(ai_file)

            if feature_name not in ai_df.columns:
                print(f"  Warning: '{feature_name}' not found in {ai_file}, skipping")
                continue

            ai_feature_data = ai_df[[feature_name]].copy()
            ai_feature_data.columns = [feature_name]

            # Load Human data
            human_file = file_mapping[source]['human']
            human_df = load_excel_data(human_file)

            if feature_name not in human_df.columns:
                print(f"  Warning: '{feature_name}' not found in {human_file}, skipping")
                continue

            human_feature_data = human_df[[feature_name]].copy()
            human_feature_data.columns = [feature_name]

            ai_features.append(ai_feature_data)
            human_features.append(human_feature_data)

            print(f"  ✓ Successfully loaded from both AI and Human files")

        except Exception as e:
            print(f"  Error loading {feature_name} from {source}: {e}")
            continue

    # Combine all features
    print("\nCombining features...")
    ai_combined = pd.concat(ai_features, axis=1)
    human_combined = pd.concat(human_features, axis=1)

    # Add labels
    ai_combined['label'] = AI_LABEL
    human_combined['label'] = HUMAN_LABEL

    # Combine AI and Human
    final_df = pd.concat([ai_combined, human_combined], axis=0, ignore_index=True)

    print(f"\n✓ Dataset built successfully!")
    print(f"  Shape: {final_df.shape}")
    print(f"  Features: {final_df.shape[1] - 1}")
    print(f"  Samples: {final_df.shape[0]}")
    print(f"  AI samples: {(final_df['label'] == AI_LABEL).sum()}")
    print(f"  Human samples: {(final_df['label'] == HUMAN_LABEL).sum()}")

    return final_df

# ============================================================================
# PREPROCESSING FUNCTIONS
# ============================================================================

def preprocess_data(df):
    """Handle missing values and prepare data"""
    print("\n" + "=" * 80)
    print("PREPROCESSING DATA")
    print("=" * 80)

    # Separate features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    print(f"Initial shape: {X.shape}")

    # Handle -1 values (replace with column median or 0)
    print("\nHandling missing values (-1)...")
    for col in X.columns:
        missing_count = (X[col] == -1).sum()
        if missing_count > 0:
            print(f"  {col}: {missing_count} missing values")
            # Replace -1 with median (or 0 if all are -1)
            median_val = X[X[col] != -1][col].median()
            if pd.isna(median_val):
                X[col] = X[col].replace(-1, 0)
            else:
                X[col] = X[col].replace(-1, median_val)

    # Handle boolean columns (convert True/False to 1/0)
    bool_cols = X.select_dtypes(include=['bool']).columns
    if len(bool_cols) > 0:
        print(f"\nConverting boolean columns to numeric: {list(bool_cols)}")
        X[bool_cols] = X[bool_cols].astype(int)

    # Check for any remaining NaN or inf values
    if X.isnull().any().any():
        print("\nWarning: NaN values found, filling with 0")
        X = X.fillna(0)

    if np.isinf(X.values).any():
        print("\nWarning: Infinite values found, replacing with column max")
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.max())

    print(f"\n✓ Preprocessing complete!")
    print(f"  Final shape: {X.shape}")
    print(f"  Class distribution:")
    print(f"    AI (0): {(y == 0).sum()}")
    print(f"    Human (1): {(y == 1).sum()}")

    return X, y

def split_and_scale_data(X, y):
    """Split data and scale features"""
    print("\n" + "=" * 80)
    print("SPLITTING AND SCALING DATA")
    print("=" * 80)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
    )

    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")

    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

    print("✓ Scaling complete (StandardScaler)")

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# ============================================================================
# MODEL DEFINITIONS
# ============================================================================

def get_all_models():
    """Return dictionary of all models to benchmark"""
    models = {
        'Logistic Regression': LogisticRegression(random_state=RANDOM_SEED, max_iter=1000),
        'Ridge Classifier': RidgeClassifier(random_state=RANDOM_SEED),
        'SGD Classifier': SGDClassifier(random_state=RANDOM_SEED, max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_SEED),
        'Random Forest': RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Extra Trees': ExtraTreesClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_SEED),
        'KNN': KNeighborsClassifier(),
        'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=RANDOM_SEED),
        'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=RANDOM_SEED),
        'Gaussian NB': GaussianNB(),
    }

    # Add XGBoost if available
    if XGBOOST_AVAILABLE:
        models['XGBoost'] = xgb.XGBClassifier(random_state=RANDOM_SEED, eval_metric='logloss')

    # Add LightGBM if available
    if LIGHTGBM_AVAILABLE:
        models['LightGBM'] = lgb.LGBMClassifier(random_state=RANDOM_SEED, verbose=-1)

    # Add CatBoost if available
    if CATBOOST_AVAILABLE:
        models['CatBoost'] = cb.CatBoostClassifier(random_state=RANDOM_SEED, verbose=0)

    return models

# ============================================================================
# TRAINING AND EVALUATION
# ============================================================================

def train_and_evaluate_model(name, model, X_train, X_test, y_train, y_test):
    """Train and evaluate a single model"""
    print(f"\n{'─' * 80}")
    print(f"Training: {name}")
    print(f"{'─' * 80}")

    results = {'Model': name}

    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    results['Train Time (s)'] = round(train_time, 3)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Metrics
    results['Accuracy'] = accuracy_score(y_test, y_pred)
    results['Precision'] = precision_score(y_test, y_pred)
    results['Recall'] = recall_score(y_test, y_pred)
    results['F1-Score'] = f1_score(y_test, y_pred)

    if y_pred_proba is not None:
        results['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)
    else:
        results['ROC-AUC'] = np.nan

    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=CV_FOLDS, scoring='accuracy')
    results['CV Mean'] = cv_scores.mean()
    results['CV Std'] = cv_scores.std()

    # Print results
    print(f"Accuracy: {results['Accuracy']:.4f}")
    print(f"Precision: {results['Precision']:.4f}")
    print(f"Recall: {results['Recall']:.4f}")
    print(f"F1-Score: {results['F1-Score']:.4f}")
    print(f"ROC-AUC: {results['ROC-AUC']:.4f}")
    print(f"CV Accuracy: {results['CV Mean']:.4f} (+/- {results['CV Std']:.4f})")
    print(f"Training Time: {results['Train Time (s)']} seconds")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return results, model, y_pred, y_pred_proba, cm

def benchmark_all_models(X_train, X_test, y_train, y_test):
    """Train and evaluate all models"""
    print("\n" + "=" * 80)
    print("BENCHMARKING ALL MODELS")
    print("=" * 80)

    models = get_all_models()
    results_list = []
    trained_models = {}
    predictions = {}
    confusion_matrices = {}

    for name, model in models.items():
        try:
            results, trained_model, y_pred, y_pred_proba, cm = train_and_evaluate_model(
                name, model, X_train, X_test, y_train, y_test
            )
            results_list.append(results)
            trained_models[name] = trained_model
            predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
            confusion_matrices[name] = cm

        except Exception as e:
            print(f"Error training {name}: {e}")
            continue

    # Create results DataFrame
    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values('F1-Score', ascending=False).reset_index(drop=True)

    print("\n" + "=" * 80)
    print("BENCHMARKING COMPLETE - RESULTS SUMMARY")
    print("=" * 80)
    print(results_df.to_string(index=False))

    # Save results
    results_file = os.path.join(OUTPUT_DIR, 'model_comparison_results.xlsx')
    results_df.to_excel(results_file, index=False)
    print(f"\n✓ Results saved to: {results_file}")

    return results_df, trained_models, predictions, confusion_matrices

# ============================================================================
# HYPERPARAMETER TUNING
# ============================================================================

def get_param_grids():
    """Define parameter grids for hyperparameter tuning"""
    param_grids = {
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0]
        },
        'LightGBM': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 10, 15],
            'learning_rate': [0.01, 0.1, 0.3],
            'num_leaves': [31, 50, 70]
        },
        'Logistic Regression': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'solver': ['lbfgs', 'liblinear']
        },
        'SVM (RBF)': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01]
        },
        'Gradient Boosting': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.3],
            'max_depth': [3, 5, 7]
        }
    }

    return param_grids

def tune_top_models(top_model_names, trained_models, X_train, y_train):
    """Perform hyperparameter tuning on top models"""
    print("\n" + "=" * 80)
    print(f"HYPERPARAMETER TUNING - TOP {len(top_model_names)} MODELS")
    print("=" * 80)

    param_grids = get_param_grids()
    tuned_models = {}
    tuning_results = []

    for name in top_model_names:
        if name not in param_grids:
            print(f"\nNo parameter grid defined for {name}, skipping tuning")
            tuned_models[name] = trained_models[name]
            continue

        print(f"\n{'─' * 80}")
        print(f"Tuning: {name}")
        print(f"{'─' * 80}")

        base_model = trained_models[name]
        param_grid = param_grids[name]

        # Use RandomizedSearchCV for faster tuning
        search = RandomizedSearchCV(
            base_model, param_grid, n_iter=20, cv=CV_FOLDS,
            scoring='f1', random_state=RANDOM_SEED, n_jobs=-1, verbose=1
        )

        start_time = time.time()
        search.fit(X_train, y_train)
        tune_time = time.time() - start_time

        print(f"\nBest parameters: {search.best_params_}")
        print(f"Best CV F1-Score: {search.best_score_:.4f}")
        print(f"Tuning time: {tune_time:.2f} seconds")

        tuned_models[name] = search.best_estimator_

        tuning_results.append({
            'Model': name,
            'Best CV F1-Score': search.best_score_,
            'Best Parameters': str(search.best_params_),
            'Tuning Time (s)': round(tune_time, 2)
        })

    # Save tuning results
    tuning_df = pd.DataFrame(tuning_results)
    tuning_file = os.path.join(OUTPUT_DIR, 'top_models_tuning_results.xlsx')
    tuning_df.to_excel(tuning_file, index=False)
    print(f"\n✓ Tuning results saved to: {tuning_file}")

    return tuned_models

# ============================================================================
# FINAL EVALUATION
# ============================================================================

def final_evaluation(tuned_models, X_train, X_test, y_train, y_test):
    """Evaluate tuned models on test set"""
    print("\n" + "=" * 80)
    print("FINAL EVALUATION OF TUNED MODELS")
    print("=" * 80)

    final_results = []
    final_predictions = {}
    final_cms = {}

    for name, model in tuned_models.items():
        print(f"\nEvaluating: {name}")

        # Predict
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

        # Metrics
        results = {
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1-Score': f1_score(y_test, y_pred),
            'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else np.nan
        }

        final_results.append(results)
        final_predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
        final_cms[name] = confusion_matrix(y_test, y_pred)

        print(f"  Accuracy: {results['Accuracy']:.4f}")
        print(f"  F1-Score: {results['F1-Score']:.4f}")
        print(f"  ROC-AUC: {results['ROC-AUC']:.4f}")

    final_df = pd.DataFrame(final_results).sort_values('F1-Score', ascending=False)

    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(final_df.to_string(index=False))

    # Save final results
    final_file = os.path.join(OUTPUT_DIR, 'final_tuned_models_results.xlsx')
    final_df.to_excel(final_file, index=False)
    print(f"\n✓ Final results saved to: {final_file}")

    # Get best model
    best_model_name = final_df.iloc[0]['Model']
    best_model = tuned_models[best_model_name]

    print(f"\n🏆 BEST MODEL: {best_model_name}")
    print(f"   F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"   Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")

    # Save best model
    model_file = os.path.join(OUTPUT_DIR, f'best_model_{best_model_name.replace(" ", "_")}.pkl')
    joblib.dump(best_model, model_file)
    print(f"\n✓ Best model saved to: {model_file}")

    return final_df, best_model, best_model_name, final_predictions, final_cms

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_model_comparison(results_df, save_path):
    """Plot model performance comparison"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

    for idx, metric in enumerate(metrics):
        ax = axes[idx // 2, idx % 2]
        data = results_df.sort_values(metric, ascending=True)

        ax.barh(data['Model'], data[metric], color='steelblue')
        ax.set_xlabel(metric, fontsize=12)
        ax.set_title(f'{metric} by Model', fontsize=12, fontweight='bold')
        ax.set_xlim([0, 1])

        # Add value labels
        for i, v in enumerate(data[metric]):
            ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Comparison plot saved to: {save_path}")
    plt.close()

def plot_confusion_matrices(confusion_matrices, save_path):
    """Plot confusion matrices for all models"""
    n_models = len(confusion_matrices)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_models > 1 else [axes]

    for idx, (name, cm) in enumerate(confusion_matrices.items()):
        ax = axes[idx]
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                   xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
        ax.set_title(f'{name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('True Label')
        ax.set_xlabel('Predicted Label')

    # Hide unused subplots
    for idx in range(n_models, len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrices saved to: {save_path}")
    plt.close()

def plot_roc_curves(predictions, y_test, save_path):
    """Plot ROC curves for all models"""
    plt.figure(figsize=(12, 8))

    for name, preds in predictions.items():
        if preds['y_pred_proba'] is not None:
            fpr, tpr, _ = roc_curve(y_test, preds['y_pred_proba'])
            auc = roc_auc_score(y_test, preds['y_pred_proba'])
            plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - All Models', fontsize=14, fontweight='bold')
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ ROC curves saved to: {save_path}")
    plt.close()

def plot_feature_importance(model, feature_names, model_name, save_path):
    """Plot feature importance for tree-based models"""
    if not hasattr(model, 'feature_importances_'):
        print(f"Feature importance not available for {model_name}")
        return

    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:20]  # Top 20

    plt.figure(figsize=(12, 8))
    plt.barh(range(len(indices)), importances[indices], color='steelblue')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title(f'Top 20 Feature Importance - {model_name}', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Feature importance plot saved to: {save_path}")
    plt.close()

    # Save to Excel
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

    excel_path = save_path.replace('.png', '.xlsx')
    importance_df.to_excel(excel_path, index=False)
    print(f"✓ Feature importance data saved to: {excel_path}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution pipeline"""

    # Step 1: Load feature recommendations
    feature_source_map = load_feature_recommendations(BEST_FEATURES_FILE)

    # Step 2: Build mixed-source dataset
    df = build_mixed_source_dataset(feature_source_map, FILE_MAPPING)

    # Step 3: Preprocess data
    X, y = preprocess_data(df)

    # Step 4: Split and scale
    X_train, X_test, y_train, y_test, scaler = split_and_scale_data(X, y)

    # Step 5: Benchmark all models
    results_df, trained_models, predictions, confusion_matrices = benchmark_all_models(
        X_train, X_test, y_train, y_test
    )

    # Step 6: Visualizations for all models
    print("\n" + "=" * 80)
    print("GENERATING VISUALIZATIONS")
    print("=" * 80)

    plot_model_comparison(
        results_df,
        os.path.join(OUTPUT_DIR, 'model_comparison.png')
    )

    plot_confusion_matrices(
        confusion_matrices,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_all.png')
    )

    plot_roc_curves(
        predictions,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_all.png')
    )

    # Step 7: Select top models for tuning
    top_models = results_df.head(TOP_N_MODELS_TO_TUNE)['Model'].tolist()
    print(f"\n" + "=" * 80)
    print(f"TOP {TOP_N_MODELS_TO_TUNE} MODELS FOR HYPERPARAMETER TUNING:")
    for i, model_name in enumerate(top_models, 1):
        print(f"  {i}. {model_name}")
    print("=" * 80)

    # Step 8: Hyperparameter tuning
    tuned_models = tune_top_models(top_models, trained_models, X_train, y_train)

    # Step 9: Final evaluation
    final_df, best_model, best_model_name, final_preds, final_cms = final_evaluation(
        tuned_models, X_train, X_test, y_train, y_test
    )

    # Step 10: Final visualizations
    plot_confusion_matrices(
        final_cms,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_tuned.png')
    )

    plot_roc_curves(
        final_preds,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_tuned.png')
    )

    # Step 11: Feature importance for best model
    plot_feature_importance(
        best_model,
        X.columns.tolist(),
        best_model_name,
        os.path.join(OUTPUT_DIR, f'feature_importance_{best_model_name.replace(" ", "_")}.png')
    )

    # Step 12: Generate classification report for best model
    print("\n" + "=" * 80)
    print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
    print("=" * 80)
    y_pred_best = best_model.predict(X_test)
    print(classification_report(y_test, y_pred_best, target_names=['AI', 'Human']))

    # Save classification report
    report_dict = classification_report(y_test, y_pred_best, target_names=['AI', 'Human'], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_file = os.path.join(OUTPUT_DIR, f'classification_report_{best_model_name.replace(" ", "_")}.xlsx')
    report_df.to_excel(report_file)
    print(f"\n✓ Classification report saved to: {report_file}")

    # Save scaler
    scaler_file = os.path.join(OUTPUT_DIR, 'scaler.pkl')
    joblib.dump(scaler, scaler_file)
    print(f"✓ Scaler saved to: {scaler_file}")

    print("\n" + "=" * 80)
    print("🎉 ML BENCHMARKING PIPELINE COMPLETE!")
    print("=" * 80)
    print(f"📁 All results saved to: {OUTPUT_DIR}/")
    print(f"🏆 Best Model: {best_model_name}")
    print(f"📊 F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"📊 Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")
    print("=" * 80)

if __name__ == "__main__":
    main()

AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM
Class Labels: AI = 0, Human = 1
Test Size: 0.2, CV Folds: 5, Random Seed: 42

Loading feature recommendations from: best_features_summary.xlsx
Total features in file: 21
Highly recommended features: 16

Highly Recommended Features and Their Sources:
  - transitivity: nltk
  - top_eigenvalue: nltk_lemma
  - avg_closeness_centrality: nltk
  - avg_degree_centrality: nltk
  - density: nltk
  - algebraic_connectivity: nltk_lemma
  - matching_number: nltk
  - size_largest_component: nltk
  - number_of_nodes: nltk
  - graph_energy: nltk
  - wiener_index: nltk
  - avg_shortest_path_length: spacy
  - radius: spacy
  - diameter: simple
  - second_eigenvalue: nltk
  - avg_betweenness_centrality: spacy

BUILDING MIXED-SOURCE DATASET

Processing feature: transitivity from source: nltk
  ✓ Successfully loaded from both AI and Human files

Processing feature: top_eigenvalue from source: nltk_lemma
  ✓ Successfully loaded from both AI and Human 

In [3]:
import pandas as pd
import numpy as np
import warnings
import time
import os
from collections import defaultdict

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, roc_curve)

# Models
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, VotingClassifier, StackingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Advanced models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("Warning: XGBoost not available")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("Warning: LightGBM not available")

try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("Warning: CatBoost not available")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# CONFIGURATION
# ============================================================================

# FILE PATHS - UPDATE THESE WITH YOUR ACTUAL FILE PATHS
FILE_MAPPING = {
    'simple': {
        'ai': 'AI_english_pmi_graph_analysis_sp_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_sp_nolemma.xlsx'
    },
    'simple_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_sp_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_sp_lemma.xlsx'
    },
    'nltk': {
        'ai': 'AI_english_pmi_graph_analysis_adv_nltk_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_nltk_nolemma.xlsx'
    },
    'nltk_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_adv_nltk_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_nltk_lemma.xlsx'
    },
    'spacy': {
        'ai': 'AI_english_pmi_graph_analysis_adv_spacy_nolemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_spacy_nolemma.xlsx'
    },
    'spacy_lemma': {
        'ai': 'AI_english_pmi_graph_analysis_adv_spacy_lemma.xlsx',
        'human': 'Human_english_pmi_graph_analysis_adv_spacy_lemma.xlsx'
    }
}

# Statistical test results file
BEST_FEATURES_FILE = 'best_features_summary.xlsx'

# Output directory
OUTPUT_DIR = 'ml_classification_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Parameters
TEST_SIZE = 0.2
RANDOM_SEED = 42
CV_FOLDS = 5
TOP_N_MODELS_TO_TUNE = 10

# Class labels (documented clearly)
# 0 = AI-generated text
# 1 = Human-written text
AI_LABEL = 0
HUMAN_LABEL = 1

print("=" * 80)
print("AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM")
print("=" * 80)
print(f"Class Labels: AI = {AI_LABEL}, Human = {HUMAN_LABEL}")
print(f"Test Size: {TEST_SIZE}, CV Folds: {CV_FOLDS}, Random Seed: {RANDOM_SEED}")
print("=" * 80 + "\n")

# ============================================================================
# SOURCE NAME MAPPING
# ============================================================================

def map_source_name(source_str):
    """Map source names from Excel to file mapping keys"""
    mapping = {
        'Spacy Token': 'spacy',
        'Spacy Lemma': 'spacy_lemma',
        'Simple Token': 'simple',
        'Simple Lemma': 'simple_lemma',
        'Nltk Token': 'nltk',
        'Nltk Lemma': 'nltk_lemma'
    }
    return mapping.get(source_str, source_str.lower().replace(' ', '_'))

# ============================================================================
# DATA LOADING FUNCTIONS
# ============================================================================

def load_feature_recommendations(filepath):
    """Load and filter highly recommended features with their sources"""
    print(f"Loading feature recommendations from: {filepath}")

    df = pd.read_excel(filepath)

    # Filter only highly recommended features
    highly_recommended = df[df['feature_recommendation'] == 'Highly Recommended'].copy()

    print(f"Total features in file: {len(df)}")
    print(f"Highly recommended features: {len(highly_recommended)}")

    # Create feature-source mapping
    feature_source_map = {}
    for _, row in highly_recommended.iterrows():
        feature_name = row['metric']
        source = map_source_name(row['source'])
        feature_source_map[feature_name] = source

    print(f"\nHighly Recommended Features and Their Sources:")
    for feature, source in feature_source_map.items():
        print(f"  - {feature}: {source}")

    return feature_source_map

def load_excel_data(filepath, max_rows=300):
    """Load Excel file and clean it"""
    df = pd.read_excel(filepath)

    # Remove 'Average' row if exists
    df = df[df['Graph'] != 'Average'].copy()

    # Remove 'Graph' column
    if 'Graph' in df.columns:
        df = df.drop('Graph', axis=1)

    # Keep only first max_rows
    df = df.head(max_rows)

    return df

def build_mixed_source_dataset(feature_source_map, file_mapping):
    """
    Build dataset by selecting each feature from its best-performing source
    """
    print("\n" + "=" * 80)
    print("BUILDING MIXED-SOURCE DATASET")
    print("=" * 80)

    ai_features = []
    human_features = []

    for feature_name, source in feature_source_map.items():
        print(f"\nProcessing feature: {feature_name} from source: {source}")

        try:
            # Load AI data
            ai_file = file_mapping[source]['ai']
            ai_df = load_excel_data(ai_file)

            if feature_name not in ai_df.columns:
                print(f"  Warning: '{feature_name}' not found in {ai_file}, skipping")
                continue

            ai_feature_data = ai_df[[feature_name]].copy()
            ai_feature_data.columns = [feature_name]

            # Load Human data
            human_file = file_mapping[source]['human']
            human_df = load_excel_data(human_file)

            if feature_name not in human_df.columns:
                print(f"  Warning: '{feature_name}' not found in {human_file}, skipping")
                continue

            human_feature_data = human_df[[feature_name]].copy()
            human_feature_data.columns = [feature_name]

            ai_features.append(ai_feature_data)
            human_features.append(human_feature_data)

            print(f"  ✓ Successfully loaded from both AI and Human files")

        except Exception as e:
            print(f"  Error loading {feature_name} from {source}: {e}")
            continue

    # Combine all features
    print("\nCombining features...")
    ai_combined = pd.concat(ai_features, axis=1)
    human_combined = pd.concat(human_features, axis=1)

    # Add labels
    ai_combined['label'] = AI_LABEL
    human_combined['label'] = HUMAN_LABEL

    # Combine AI and Human
    final_df = pd.concat([ai_combined, human_combined], axis=0, ignore_index=True)

    print(f"\n✓ Dataset built successfully!")
    print(f"  Shape: {final_df.shape}")
    print(f"  Features: {final_df.shape[1] - 1}")
    print(f"  Samples: {final_df.shape[0]}")
    print(f"  AI samples: {(final_df['label'] == AI_LABEL).sum()}")
    print(f"  Human samples: {(final_df['label'] == HUMAN_LABEL).sum()}")

    return final_df

# ============================================================================
# PREPROCESSING FUNCTIONS
# ============================================================================

def preprocess_data(df):
    """Handle missing values and prepare data"""
    print("\n" + "=" * 80)
    print("PREPROCESSING DATA")
    print("=" * 80)

    # Separate features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    print(f"Initial shape: {X.shape}")

    # Handle -1 values (replace with column median or 0)
    print("\nHandling missing values (-1)...")
    for col in X.columns:
        missing_count = (X[col] == -1).sum()
        if missing_count > 0:
            print(f"  {col}: {missing_count} missing values")
            # Replace -1 with median (or 0 if all are -1)
            median_val = X[X[col] != -1][col].median()
            if pd.isna(median_val):
                X[col] = X[col].replace(-1, 0)
            else:
                X[col] = X[col].replace(-1, median_val)

    # Handle boolean columns (convert True/False to 1/0)
    bool_cols = X.select_dtypes(include=['bool']).columns
    if len(bool_cols) > 0:
        print(f"\nConverting boolean columns to numeric: {list(bool_cols)}")
        X[bool_cols] = X[bool_cols].astype(int)

    # Check for any remaining NaN or inf values
    if X.isnull().any().any():
        print("\nWarning: NaN values found, filling with 0")
        X = X.fillna(0)

    if np.isinf(X.values).any():
        print("\nWarning: Infinite values found, replacing with column max")
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.max())

    print(f"\n✓ Preprocessing complete!")
    print(f"  Final shape: {X.shape}")
    print(f"  Class distribution:")
    print(f"    AI (0): {(y == 0).sum()}")
    print(f"    Human (1): {(y == 1).sum()}")

    return X, y

def split_and_scale_data(X, y):
    """Split data and scale features"""
    print("\n" + "=" * 80)
    print("SPLITTING AND SCALING DATA")
    print("=" * 80)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
    )

    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")

    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

    print("✓ Scaling complete (StandardScaler)")

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# ============================================================================
# MODEL DEFINITIONS
# ============================================================================

def get_all_models():
    """Return dictionary of all models to benchmark"""
    models = {
        'Logistic Regression': LogisticRegression(random_state=RANDOM_SEED, max_iter=1000),
        'Ridge Classifier': RidgeClassifier(random_state=RANDOM_SEED),
        'SGD Classifier': SGDClassifier(random_state=RANDOM_SEED, max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_SEED),
        'Random Forest': RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Extra Trees': ExtraTreesClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_SEED),
        'KNN': KNeighborsClassifier(),
        'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=RANDOM_SEED),
        'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=RANDOM_SEED),
        'Gaussian NB': GaussianNB(),
    }

    # Add XGBoost if available
    if XGBOOST_AVAILABLE:
        models['XGBoost'] = xgb.XGBClassifier(random_state=RANDOM_SEED, eval_metric='logloss')

    # Add LightGBM if available
    if LIGHTGBM_AVAILABLE:
        models['LightGBM'] = lgb.LGBMClassifier(random_state=RANDOM_SEED, verbose=-1)

    # Add CatBoost if available
    if CATBOOST_AVAILABLE:
        models['CatBoost'] = cb.CatBoostClassifier(random_state=RANDOM_SEED, verbose=0)

    return models

# ============================================================================
# TRAINING AND EVALUATION
# ============================================================================

def train_and_evaluate_model(name, model, X_train, X_test, y_train, y_test):
    """Train and evaluate a single model"""
    print(f"\n{'─' * 80}")
    print(f"Training: {name}")
    print(f"{'─' * 80}")

    results = {'Model': name}

    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    results['Train Time (s)'] = round(train_time, 3)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Metrics
    results['Accuracy'] = accuracy_score(y_test, y_pred)
    results['Precision'] = precision_score(y_test, y_pred)
    results['Recall'] = recall_score(y_test, y_pred)
    results['F1-Score'] = f1_score(y_test, y_pred)

    if y_pred_proba is not None:
        results['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)
    else:
        results['ROC-AUC'] = np.nan

    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=CV_FOLDS, scoring='accuracy')
    results['CV Mean'] = cv_scores.mean()
    results['CV Std'] = cv_scores.std()

    # Print results
    print(f"Accuracy: {results['Accuracy']:.4f}")
    print(f"Precision: {results['Precision']:.4f}")
    print(f"Recall: {results['Recall']:.4f}")
    print(f"F1-Score: {results['F1-Score']:.4f}")
    print(f"ROC-AUC: {results['ROC-AUC']:.4f}")
    print(f"CV Accuracy: {results['CV Mean']:.4f} (+/- {results['CV Std']:.4f})")
    print(f"Training Time: {results['Train Time (s)']} seconds")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return results, model, y_pred, y_pred_proba, cm

def benchmark_all_models(X_train, X_test, y_train, y_test):
    """Train and evaluate all models"""
    print("\n" + "=" * 80)
    print("BENCHMARKING ALL MODELS")
    print("=" * 80)

    models = get_all_models()
    results_list = []
    trained_models = {}
    predictions = {}
    confusion_matrices = {}

    for name, model in models.items():
        try:
            results, trained_model, y_pred, y_pred_proba, cm = train_and_evaluate_model(
                name, model, X_train, X_test, y_train, y_test
            )
            results_list.append(results)
            trained_models[name] = trained_model
            predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
            confusion_matrices[name] = cm

        except Exception as e:
            print(f"Error training {name}: {e}")
            continue

    # Create results DataFrame
    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values('F1-Score', ascending=False).reset_index(drop=True)

    print("\n" + "=" * 80)
    print("BENCHMARKING COMPLETE - RESULTS SUMMARY")
    print("=" * 80)
    print(results_df.to_string(index=False))

    # Save results
    results_file = os.path.join(OUTPUT_DIR, 'model_comparison_results.xlsx')
    results_df.to_excel(results_file, index=False)
    print(f"\n✓ Results saved to: {results_file}")

    return results_df, trained_models, predictions, confusion_matrices

# ============================================================================
# HYPERPARAMETER TUNING
# ============================================================================

def get_param_grids():
    """Define parameter grids for hyperparameter tuning"""
    param_grids = {
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0]
        },
        'LightGBM': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 10, 15],
            'learning_rate': [0.01, 0.1, 0.3],
            'num_leaves': [31, 50, 70]
        },
        'Logistic Regression': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'solver': ['lbfgs', 'liblinear']
        },
        'SVM (RBF)': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01]
        },
        'Gradient Boosting': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.3],
            'max_depth': [3, 5, 7]
        }
    }

    return param_grids

def tune_top_models(top_model_names, trained_models, X_train, y_train):
    """Perform hyperparameter tuning on top models"""
    print("\n" + "=" * 80)
    print(f"HYPERPARAMETER TUNING - TOP {len(top_model_names)} MODELS")
    print("=" * 80)

    param_grids = get_param_grids()
    tuned_models = {}
    tuning_results = []

    for name in top_model_names:
        if name not in param_grids:
            print(f"\nNo parameter grid defined for {name}, skipping tuning")
            tuned_models[name] = trained_models[name]
            continue

        print(f"\n{'─' * 80}")
        print(f"Tuning: {name}")
        print(f"{'─' * 80}")

        base_model = trained_models[name]
        param_grid = param_grids[name]

        # Use RandomizedSearchCV for faster tuning
        search = RandomizedSearchCV(
            base_model, param_grid, n_iter=20, cv=CV_FOLDS,
            scoring='f1', random_state=RANDOM_SEED, n_jobs=-1, verbose=1
        )

        start_time = time.time()
        search.fit(X_train, y_train)
        tune_time = time.time() - start_time

        print(f"\nBest parameters: {search.best_params_}")
        print(f"Best CV F1-Score: {search.best_score_:.4f}")
        print(f"Tuning time: {tune_time:.2f} seconds")

        tuned_models[name] = search.best_estimator_

        tuning_results.append({
            'Model': name,
            'Best CV F1-Score': search.best_score_,
            'Best Parameters': str(search.best_params_),
            'Tuning Time (s)': round(tune_time, 2)
        })

    # Save tuning results
    tuning_df = pd.DataFrame(tuning_results)
    tuning_file = os.path.join(OUTPUT_DIR, 'top_models_tuning_results.xlsx')
    tuning_df.to_excel(tuning_file, index=False)
    print(f"\n✓ Tuning results saved to: {tuning_file}")

    return tuned_models

# ============================================================================
# FINAL EVALUATION
# ============================================================================

def final_evaluation(tuned_models, X_train, X_test, y_train, y_test):
    """Evaluate tuned models on test set"""
    print("\n" + "=" * 80)
    print("FINAL EVALUATION OF TUNED MODELS")
    print("=" * 80)

    final_results = []
    final_predictions = {}
    final_cms = {}

    for name, model in tuned_models.items():
        print(f"\nEvaluating: {name}")

        # Predict
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

        # Metrics
        results = {
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1-Score': f1_score(y_test, y_pred),
            'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else np.nan
        }

        final_results.append(results)
        final_predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
        final_cms[name] = confusion_matrix(y_test, y_pred)

        print(f"  Accuracy: {results['Accuracy']:.4f}")
        print(f"  F1-Score: {results['F1-Score']:.4f}")
        print(f"  ROC-AUC: {results['ROC-AUC']:.4f}")

    final_df = pd.DataFrame(final_results).sort_values('F1-Score', ascending=False)

    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(final_df.to_string(index=False))

    # Save final results
    final_file = os.path.join(OUTPUT_DIR, 'final_tuned_models_results.xlsx')
    final_df.to_excel(final_file, index=False)
    print(f"\n✓ Final results saved to: {final_file}")

    # Get best model
    best_model_name = final_df.iloc[0]['Model']
    best_model = tuned_models[best_model_name]

    print(f"\n🏆 BEST MODEL: {best_model_name}")
    print(f"   F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"   Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")

    # Save best model
    model_file = os.path.join(OUTPUT_DIR, f'best_model_{best_model_name.replace(" ", "_")}.pkl')
    joblib.dump(best_model, model_file)
    print(f"\n✓ Best model saved to: {model_file}")

    return final_df, best_model, best_model_name, final_predictions, final_cms

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_model_comparison(results_df, save_path):
    """Plot model performance comparison"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

    for idx, metric in enumerate(metrics):
        ax = axes[idx // 2, idx % 2]
        data = results_df.sort_values(metric, ascending=True)

        ax.barh(data['Model'], data[metric], color='steelblue')
        ax.set_xlabel(metric, fontsize=12)
        ax.set_title(f'{metric} by Model', fontsize=12, fontweight='bold')
        ax.set_xlim([0, 1])

        # Add value labels
        for i, v in enumerate(data[metric]):
            ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Comparison plot saved to: {save_path}")
    plt.close()

def plot_confusion_matrices(confusion_matrices, save_path):
    """Plot confusion matrices for all models"""
    n_models = len(confusion_matrices)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_models > 1 else [axes]

    for idx, (name, cm) in enumerate(confusion_matrices.items()):
        ax = axes[idx]
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                   xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
        ax.set_title(f'{name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('True Label')
        ax.set_xlabel('Predicted Label')

    # Hide unused subplots
    for idx in range(n_models, len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrices saved to: {save_path}")
    plt.close()

def plot_roc_curves(predictions, y_test, save_path):
    """Plot ROC curves for all models"""
    plt.figure(figsize=(12, 8))

    for name, preds in predictions.items():
        if preds['y_pred_proba'] is not None:
            fpr, tpr, _ = roc_curve(y_test, preds['y_pred_proba'])
            auc = roc_auc_score(y_test, preds['y_pred_proba'])
            plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - All Models', fontsize=14, fontweight='bold')
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ ROC curves saved to: {save_path}")
    plt.close()

def plot_feature_importance(model, feature_names, model_name, save_path):
    """Plot feature importance for tree-based models"""
    if not hasattr(model, 'feature_importances_'):
        print(f"Feature importance not available for {model_name}")
        return

    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:20]  # Top 20

    plt.figure(figsize=(12, 8))
    plt.barh(range(len(indices)), importances[indices], color='steelblue')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title(f'Top 20 Feature Importance - {model_name}', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Feature importance plot saved to: {save_path}")
    plt.close()

    # Save to Excel
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

    excel_path = save_path.replace('.png', '.xlsx')
    importance_df.to_excel(excel_path, index=False)
    print(f"✓ Feature importance data saved to: {excel_path}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution pipeline"""

    # Step 1: Load feature recommendations
    feature_source_map = load_feature_recommendations(BEST_FEATURES_FILE)

    # Step 2: Build mixed-source dataset
    df = build_mixed_source_dataset(feature_source_map, FILE_MAPPING)

    # ========================================================================
    # SAVE FINAL COMBINED DATASET - ADD THIS CODE HERE
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING FINAL COMBINED DATASET")
    print("=" * 80)

    # Save the complete dataset (before preprocessing)
    combined_dataset_file = os.path.join(OUTPUT_DIR, 'final_combined_dataset.xlsx')
    df.to_excel(combined_dataset_file, index=False)
    print(f"✓ Final combined dataset saved to: {combined_dataset_file}")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

    # Also save as CSV for easier access
    combined_dataset_csv = os.path.join(OUTPUT_DIR, 'final_combined_dataset.csv')
    df.to_csv(combined_dataset_csv, index=False)
    print(f"✓ Final combined dataset (CSV) saved to: {combined_dataset_csv}")

    # Save a summary of data sources for each feature
    feature_source_summary = pd.DataFrame([
        {'Feature': feature, 'Source': source}
        for feature, source in feature_source_map.items()
    ])
    source_summary_file = os.path.join(OUTPUT_DIR, 'feature_source_mapping.xlsx')
    feature_source_summary.to_excel(source_summary_file, index=False)
    print(f"✓ Feature-source mapping saved to: {source_summary_file}")
    print("=" * 80)
    # ========================================================================

    # Step 3: Preprocess data
    X, y = preprocess_data(df)

    # Step 4: Split and scale
    X_train, X_test, y_train, y_test, scaler = split_and_scale_data(X, y)

    # ========================================================================
    # SAVE PREPROCESSED DATASETS - ADD THIS CODE HERE (OPTIONAL)
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING PREPROCESSED DATASETS")
    print("=" * 80)

    # Save train and test sets (before scaling)
    train_df = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
    test_df = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

    train_file = os.path.join(OUTPUT_DIR, 'train_set.xlsx')
    test_file = os.path.join(OUTPUT_DIR, 'test_set.xlsx')

    train_df.to_excel(train_file, index=False)
    test_df.to_excel(test_file, index=False)

    print(f"✓ Training set saved to: {train_file}")
    print(f"  Shape: {train_df.shape}")
    print(f"✓ Test set saved to: {test_file}")
    print(f"  Shape: {test_df.shape}")
    print("=" * 80)
    # ========================================================================

    # Step 5: Benchmark all models
    results_df, trained_models, predictions, confusion_matrices = benchmark_all_models(
        X_train, X_test, y_train, y_test
    )

    # Step 6: Visualizations for all models
    print("\n" + "=" * 80)
    print("GENERATING VISUALIZATIONS")
    print("=" * 80)

    plot_model_comparison(
        results_df,
        os.path.join(OUTPUT_DIR, 'model_comparison.png')
    )

    plot_confusion_matrices(
        confusion_matrices,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_all.png')
    )

    plot_roc_curves(
        predictions,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_all.png')
    )

    # Step 7: Select top models for tuning
    top_models = results_df.head(TOP_N_MODELS_TO_TUNE)['Model'].tolist()
    print(f"\n" + "=" * 80)
    print(f"TOP {TOP_N_MODELS_TO_TUNE} MODELS FOR HYPERPARAMETER TUNING:")
    for i, model_name in enumerate(top_models, 1):
        print(f"  {i}. {model_name}")
    print("=" * 80)

    # Step 8: Hyperparameter tuning
    tuned_models = tune_top_models(top_models, trained_models, X_train, y_train)

    # Step 9: Final evaluation
    final_df, best_model, best_model_name, final_preds, final_cms = final_evaluation(
        tuned_models, X_train, X_test, y_train, y_test
    )

    # Step 10: Final visualizations
    plot_confusion_matrices(
        final_cms,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_tuned.png')
    )

    plot_roc_curves(
        final_preds,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_tuned.png')
    )

    # Step 11: Feature importance for best model
    plot_feature_importance(
        best_model,
        X.columns.tolist(),
        best_model_name,
        os.path.join(OUTPUT_DIR, f'feature_importance_{best_model_name.replace(" ", "_")}.png')
    )

    # Step 12: Generate classification report for best model
    print("\n" + "=" * 80)
    print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
    print("=" * 80)
    y_pred_best = best_model.predict(X_test)
    print(classification_report(y_test, y_pred_best, target_names=['AI', 'Human']))

    # Save classification report
    report_dict = classification_report(y_test, y_pred_best, target_names=['AI', 'Human'], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_file = os.path.join(OUTPUT_DIR, f'classification_report_{best_model_name.replace(" ", "_")}.xlsx')
    report_df.to_excel(report_file)
    print(f"\n✓ Classification report saved to: {report_file}")

    # Save scaler
    scaler_file = os.path.join(OUTPUT_DIR, 'scaler.pkl')
    joblib.dump(scaler, scaler_file)
    print(f"✓ Scaler saved to: {scaler_file}")

    print("\n" + "=" * 80)
    print("🎉 ML BENCHMARKING PIPELINE COMPLETE!")
    print("=" * 80)
    print(f"📁 All results saved to: {OUTPUT_DIR}/")
    print(f"🏆 Best Model: {best_model_name}")
    print(f"📊 F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"📊 Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")
    print("=" * 80)

if __name__ == "__main__":
    main()

AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM
Class Labels: AI = 0, Human = 1
Test Size: 0.2, CV Folds: 5, Random Seed: 42

Loading feature recommendations from: best_features_summary.xlsx
Total features in file: 21
Highly recommended features: 16

Highly Recommended Features and Their Sources:
  - transitivity: nltk
  - top_eigenvalue: nltk_lemma
  - avg_closeness_centrality: nltk
  - avg_degree_centrality: nltk
  - density: nltk
  - algebraic_connectivity: nltk_lemma
  - matching_number: nltk
  - size_largest_component: nltk
  - number_of_nodes: nltk
  - graph_energy: nltk
  - wiener_index: nltk
  - avg_shortest_path_length: spacy
  - radius: spacy
  - diameter: simple
  - second_eigenvalue: nltk
  - avg_betweenness_centrality: spacy

BUILDING MIXED-SOURCE DATASET

Processing feature: transitivity from source: nltk
  ✓ Successfully loaded from both AI and Human files

Processing feature: top_eigenvalue from source: nltk_lemma
  ✓ Successfully loaded from both AI and Human 

##Bengali

In [ ]:
import pandas as pd
import numpy as np
import warnings
import time
import os
from collections import defaultdict

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, roc_curve)

# Models
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, VotingClassifier, StackingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Advanced models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("Warning: XGBoost not available")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("Warning: LightGBM not available")

try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("Warning: CatBoost not available")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# CONFIGURATION
# ============================================================================

# FILE PATHS - UPDATE THESE WITH YOUR ACTUAL FILE PATHS
FILE_MAPPING = {
    'simple_bnlemma': {
        'ai': 'AI_simple_bnlemma_PMI.xlsx',
        'human': 'human_simple_bnlemma_PMI.xlsx'
    },
    'adv_bnlp': {
        'ai': 'AI_advanced_bnlp_PMI.xlsx',
        'human': 'human_advanced_bnlp_PMI.xlsx'
    },
    'adv_bnltk': {
        'ai': 'AI_advanced_bnltk_PMI.xlsx',
        'human': 'human_advanced_bnltk_PMI.xlsx'
    }
}

# Statistical test results file
BEST_FEATURES_FILE = 'best_features_summary.xlsx'

# Output directory
OUTPUT_DIR = 'ml_classification_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Parameters
TEST_SIZE = 0.2
RANDOM_SEED = 42
CV_FOLDS = 5
TOP_N_MODELS_TO_TUNE = 10

# Class labels (documented clearly)
# 0 = AI-generated text
# 1 = Human-written text
AI_LABEL = 0
HUMAN_LABEL = 1

print("=" * 80)
print("AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM")
print("=" * 80)
print(f"Class Labels: AI = {AI_LABEL}, Human = {HUMAN_LABEL}")
print(f"Test Size: {TEST_SIZE}, CV Folds: {CV_FOLDS}, Random Seed: {RANDOM_SEED}")
print("=" * 80 + "\n")

# ============================================================================
# SOURCE NAME MAPPING
# ============================================================================

def map_source_name(source_str):
    """Map source names from Excel to file mapping keys"""
    mapping = {
        'bnlemma': 'simple_bnlemma',
        'bnlp': 'adv_bnlp',
        'bnltk': 'adv_bnltk'
    }
    return mapping.get(source_str, source_str.lower().replace(' ', '_'))

# ============================================================================
# DATA LOADING FUNCTIONS
# ============================================================================

def load_feature_recommendations(filepath):
    """Load and filter highly recommended features with their sources"""
    print(f"Loading feature recommendations from: {filepath}")

    df = pd.read_excel(filepath)

    # Filter only highly recommended features
    highly_recommended = df[df['feature_recommendation'] == 'Highly Recommended'].copy()

    print(f"Total features in file: {len(df)}")
    print(f"Highly recommended features: {len(highly_recommended)}")

    # Create feature-source mapping
    feature_source_map = {}
    for _, row in highly_recommended.iterrows():
        feature_name = row['metric']
        source = map_source_name(row['source'])
        feature_source_map[feature_name] = source

    print(f"\nHighly Recommended Features and Their Sources:")
    for feature, source in feature_source_map.items():
        print(f"  - {feature}: {source}")

    return feature_source_map

def load_excel_data(filepath, max_rows=300):
    """Load Excel file and clean it"""
    df = pd.read_excel(filepath)

    # Remove 'Average' row if exists
    df = df[df['Graph'] != 'Average'].copy()

    # Remove 'Graph' column
    if 'Graph' in df.columns:
        df = df.drop('Graph', axis=1)

    # Keep only first max_rows
    df = df.head(max_rows)

    return df

def build_mixed_source_dataset(feature_source_map, file_mapping):
    """
    Build dataset by selecting each feature from its best-performing source
    """
    print("\n" + "=" * 80)
    print("BUILDING MIXED-SOURCE DATASET")
    print("=" * 80)

    ai_features = []
    human_features = []

    for feature_name, source in feature_source_map.items():
        print(f"\nProcessing feature: {feature_name} from source: {source}")

        try:
            # Load AI data
            ai_file = file_mapping[source]['ai']
            ai_df = load_excel_data(ai_file)

            if feature_name not in ai_df.columns:
                print(f"  Warning: '{feature_name}' not found in {ai_file}, skipping")
                continue

            ai_feature_data = ai_df[[feature_name]].copy()
            ai_feature_data.columns = [feature_name]

            # Load Human data
            human_file = file_mapping[source]['human']
            human_df = load_excel_data(human_file)

            if feature_name not in human_df.columns:
                print(f"  Warning: '{feature_name}' not found in {human_file}, skipping")
                continue

            human_feature_data = human_df[[feature_name]].copy()
            human_feature_data.columns = [feature_name]

            ai_features.append(ai_feature_data)
            human_features.append(human_feature_data)

            print(f"  ✓ Successfully loaded from both AI and Human files")

        except Exception as e:
            print(f"  Error loading {feature_name} from {source}: {e}")
            continue

    # Combine all features
    print("\nCombining features...")
    ai_combined = pd.concat(ai_features, axis=1)
    human_combined = pd.concat(human_features, axis=1)

    # Add labels
    ai_combined['label'] = AI_LABEL
    human_combined['label'] = HUMAN_LABEL

    # Combine AI and Human
    final_df = pd.concat([ai_combined, human_combined], axis=0, ignore_index=True)

    print(f"\n✓ Dataset built successfully!")
    print(f"  Shape: {final_df.shape}")
    print(f"  Features: {final_df.shape[1] - 1}")
    print(f"  Samples: {final_df.shape[0]}")
    print(f"  AI samples: {(final_df['label'] == AI_LABEL).sum()}")
    print(f"  Human samples: {(final_df['label'] == HUMAN_LABEL).sum()}")

    return final_df

# ============================================================================
# PREPROCESSING FUNCTIONS
# ============================================================================

def preprocess_data(df):
    """Handle missing values and prepare data"""
    print("\n" + "=" * 80)
    print("PREPROCESSING DATA")
    print("=" * 80)

    # Separate features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    print(f"Initial shape: {X.shape}")

    # Handle -1 values (replace with column median or 0)
    print("\nHandling missing values (-1)...")
    for col in X.columns:
        missing_count = (X[col] == -1).sum()
        if missing_count > 0:
            print(f"  {col}: {missing_count} missing values")
            # Replace -1 with median (or 0 if all are -1)
            median_val = X[X[col] != -1][col].median()
            if pd.isna(median_val):
                X[col] = X[col].replace(-1, 0)
            else:
                X[col] = X[col].replace(-1, median_val)

    # Handle boolean columns (convert True/False to 1/0)
    bool_cols = X.select_dtypes(include=['bool']).columns
    if len(bool_cols) > 0:
        print(f"\nConverting boolean columns to numeric: {list(bool_cols)}")
        X[bool_cols] = X[bool_cols].astype(int)

    # Check for any remaining NaN or inf values
    if X.isnull().any().any():
        print("\nWarning: NaN values found, filling with 0")
        X = X.fillna(0)

    if np.isinf(X.values).any():
        print("\nWarning: Infinite values found, replacing with column max")
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.max())

    print(f"\n✓ Preprocessing complete!")
    print(f"  Final shape: {X.shape}")
    print(f"  Class distribution:")
    print(f"    AI (0): {(y == 0).sum()}")
    print(f"    Human (1): {(y == 1).sum()}")

    return X, y

def split_and_scale_data(X, y):
    """Split data and scale features"""
    print("\n" + "=" * 80)
    print("SPLITTING AND SCALING DATA")
    print("=" * 80)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
    )

    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")

    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

    print("✓ Scaling complete (StandardScaler)")

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# ============================================================================
# MODEL DEFINITIONS
# ============================================================================

def get_all_models():
    """Return dictionary of all models to benchmark"""
    models = {
        'Logistic Regression': LogisticRegression(random_state=RANDOM_SEED, max_iter=1000),
        'Ridge Classifier': RidgeClassifier(random_state=RANDOM_SEED),
        'SGD Classifier': SGDClassifier(random_state=RANDOM_SEED, max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_SEED),
        'Random Forest': RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Extra Trees': ExtraTreesClassifier(random_state=RANDOM_SEED, n_estimators=100),
        'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_SEED),
        'KNN': KNeighborsClassifier(),
        'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=RANDOM_SEED),
        'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=RANDOM_SEED),
        'Gaussian NB': GaussianNB(),
    }

    # Add XGBoost if available
    if XGBOOST_AVAILABLE:
        models['XGBoost'] = xgb.XGBClassifier(random_state=RANDOM_SEED, eval_metric='logloss')

    # Add LightGBM if available
    if LIGHTGBM_AVAILABLE:
        models['LightGBM'] = lgb.LGBMClassifier(random_state=RANDOM_SEED, verbose=-1)

    # Add CatBoost if available
    if CATBOOST_AVAILABLE:
        models['CatBoost'] = cb.CatBoostClassifier(random_state=RANDOM_SEED, verbose=0)

    return models

# ============================================================================
# TRAINING AND EVALUATION
# ============================================================================

def train_and_evaluate_model(name, model, X_train, X_test, y_train, y_test):
    """Train and evaluate a single model"""
    print(f"\n{'─' * 80}")
    print(f"Training: {name}")
    print(f"{'─' * 80}")

    results = {'Model': name}

    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    results['Train Time (s)'] = round(train_time, 3)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Metrics
    results['Accuracy'] = accuracy_score(y_test, y_pred)
    results['Precision'] = precision_score(y_test, y_pred)
    results['Recall'] = recall_score(y_test, y_pred)
    results['F1-Score'] = f1_score(y_test, y_pred)

    if y_pred_proba is not None:
        results['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)
    else:
        results['ROC-AUC'] = np.nan

    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=CV_FOLDS, scoring='accuracy')
    results['CV Mean'] = cv_scores.mean()
    results['CV Std'] = cv_scores.std()

    # Print results
    print(f"Accuracy: {results['Accuracy']:.4f}")
    print(f"Precision: {results['Precision']:.4f}")
    print(f"Recall: {results['Recall']:.4f}")
    print(f"F1-Score: {results['F1-Score']:.4f}")
    print(f"ROC-AUC: {results['ROC-AUC']:.4f}")
    print(f"CV Accuracy: {results['CV Mean']:.4f} (+/- {results['CV Std']:.4f})")
    print(f"Training Time: {results['Train Time (s)']} seconds")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return results, model, y_pred, y_pred_proba, cm

def benchmark_all_models(X_train, X_test, y_train, y_test):
    """Train and evaluate all models"""
    print("\n" + "=" * 80)
    print("BENCHMARKING ALL MODELS")
    print("=" * 80)

    models = get_all_models()
    results_list = []
    trained_models = {}
    predictions = {}
    confusion_matrices = {}

    for name, model in models.items():
        try:
            results, trained_model, y_pred, y_pred_proba, cm = train_and_evaluate_model(
                name, model, X_train, X_test, y_train, y_test
            )
            results_list.append(results)
            trained_models[name] = trained_model
            predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
            confusion_matrices[name] = cm

        except Exception as e:
            print(f"Error training {name}: {e}")
            continue

    # Create results DataFrame
    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values('F1-Score', ascending=False).reset_index(drop=True)

    print("\n" + "=" * 80)
    print("BENCHMARKING COMPLETE - RESULTS SUMMARY")
    print("=" * 80)
    print(results_df.to_string(index=False))

    # Save results
    results_file = os.path.join(OUTPUT_DIR, 'model_comparison_results.xlsx')
    results_df.to_excel(results_file, index=False)
    print(f"\n✓ Results saved to: {results_file}")

    return results_df, trained_models, predictions, confusion_matrices

# ============================================================================
# HYPERPARAMETER TUNING
# ============================================================================

def get_param_grids():
    """Define parameter grids for hyperparameter tuning"""
    param_grids = {
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0]
        },
        'LightGBM': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 10, 15],
            'learning_rate': [0.01, 0.1, 0.3],
            'num_leaves': [31, 50, 70]
        },
        'Logistic Regression': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'solver': ['lbfgs', 'liblinear']
        },
        'SVM (RBF)': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01]
        },
        'Gradient Boosting': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.3],
            'max_depth': [3, 5, 7]
        }
    }

    return param_grids

def tune_top_models(top_model_names, trained_models, X_train, y_train):
    """Perform hyperparameter tuning on top models"""
    print("\n" + "=" * 80)
    print(f"HYPERPARAMETER TUNING - TOP {len(top_model_names)} MODELS")
    print("=" * 80)

    param_grids = get_param_grids()
    tuned_models = {}
    tuning_results = []

    for name in top_model_names:
        if name not in param_grids:
            print(f"\nNo parameter grid defined for {name}, skipping tuning")
            tuned_models[name] = trained_models[name]
            continue

        print(f"\n{'─' * 80}")
        print(f"Tuning: {name}")
        print(f"{'─' * 80}")

        base_model = trained_models[name]
        param_grid = param_grids[name]

        # Use RandomizedSearchCV for faster tuning
        search = RandomizedSearchCV(
            base_model, param_grid, n_iter=20, cv=CV_FOLDS,
            scoring='f1', random_state=RANDOM_SEED, n_jobs=-1, verbose=1
        )

        start_time = time.time()
        search.fit(X_train, y_train)
        tune_time = time.time() - start_time

        print(f"\nBest parameters: {search.best_params_}")
        print(f"Best CV F1-Score: {search.best_score_:.4f}")
        print(f"Tuning time: {tune_time:.2f} seconds")

        tuned_models[name] = search.best_estimator_

        tuning_results.append({
            'Model': name,
            'Best CV F1-Score': search.best_score_,
            'Best Parameters': str(search.best_params_),
            'Tuning Time (s)': round(tune_time, 2)
        })

    # Save tuning results
    tuning_df = pd.DataFrame(tuning_results)
    tuning_file = os.path.join(OUTPUT_DIR, 'top_models_tuning_results.xlsx')
    tuning_df.to_excel(tuning_file, index=False)
    print(f"\n✓ Tuning results saved to: {tuning_file}")

    return tuned_models

# ============================================================================
# FINAL EVALUATION
# ============================================================================

def final_evaluation(tuned_models, X_train, X_test, y_train, y_test):
    """Evaluate tuned models on test set"""
    print("\n" + "=" * 80)
    print("FINAL EVALUATION OF TUNED MODELS")
    print("=" * 80)

    final_results = []
    final_predictions = {}
    final_cms = {}

    for name, model in tuned_models.items():
        print(f"\nEvaluating: {name}")

        # Predict
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

        # Metrics
        results = {
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1-Score': f1_score(y_test, y_pred),
            'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else np.nan
        }

        final_results.append(results)
        final_predictions[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
        final_cms[name] = confusion_matrix(y_test, y_pred)

        print(f"  Accuracy: {results['Accuracy']:.4f}")
        print(f"  F1-Score: {results['F1-Score']:.4f}")
        print(f"  ROC-AUC: {results['ROC-AUC']:.4f}")

    final_df = pd.DataFrame(final_results).sort_values('F1-Score', ascending=False)

    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(final_df.to_string(index=False))

    # Save final results
    final_file = os.path.join(OUTPUT_DIR, 'final_tuned_models_results.xlsx')
    final_df.to_excel(final_file, index=False)
    print(f"\n✓ Final results saved to: {final_file}")

    # Get best model
    best_model_name = final_df.iloc[0]['Model']
    best_model = tuned_models[best_model_name]

    print(f"\n🏆 BEST MODEL: {best_model_name}")
    print(f"   F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"   Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")

    # Save best model
    model_file = os.path.join(OUTPUT_DIR, f'best_model_{best_model_name.replace(" ", "_")}.pkl')
    joblib.dump(best_model, model_file)
    print(f"\n✓ Best model saved to: {model_file}")

    return final_df, best_model, best_model_name, final_predictions, final_cms

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_model_comparison(results_df, save_path):
    """Plot model performance comparison"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

    for idx, metric in enumerate(metrics):
        ax = axes[idx // 2, idx % 2]
        data = results_df.sort_values(metric, ascending=True)

        ax.barh(data['Model'], data[metric], color='steelblue')
        ax.set_xlabel(metric, fontsize=12)
        ax.set_title(f'{metric} by Model', fontsize=12, fontweight='bold')
        ax.set_xlim([0, 1])

        # Add value labels
        for i, v in enumerate(data[metric]):
            ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Comparison plot saved to: {save_path}")
    plt.close()

def plot_confusion_matrices(confusion_matrices, save_path):
    """Plot confusion matrices for all models"""
    n_models = len(confusion_matrices)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_models > 1 else [axes]

    for idx, (name, cm) in enumerate(confusion_matrices.items()):
        ax = axes[idx]
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                   xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
        ax.set_title(f'{name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('True Label')
        ax.set_xlabel('Predicted Label')

    # Hide unused subplots
    for idx in range(n_models, len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrices saved to: {save_path}")
    plt.close()

def plot_roc_curves(predictions, y_test, save_path):
    """Plot ROC curves for all models"""
    plt.figure(figsize=(12, 8))

    for name, preds in predictions.items():
        if preds['y_pred_proba'] is not None:
            fpr, tpr, _ = roc_curve(y_test, preds['y_pred_proba'])
            auc = roc_auc_score(y_test, preds['y_pred_proba'])
            plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - All Models', fontsize=14, fontweight='bold')
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ ROC curves saved to: {save_path}")
    plt.close()

def plot_feature_importance(model, feature_names, model_name, save_path):
    """Plot feature importance for tree-based models"""
    if not hasattr(model, 'feature_importances_'):
        print(f"Feature importance not available for {model_name}")
        return

    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:20]  # Top 20

    plt.figure(figsize=(12, 8))
    plt.barh(range(len(indices)), importances[indices], color='steelblue')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title(f'Top 20 Feature Importance - {model_name}', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Feature importance plot saved to: {save_path}")
    plt.close()

    # Save to Excel
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

    excel_path = save_path.replace('.png', '.xlsx')
    importance_df.to_excel(excel_path, index=False)
    print(f"✓ Feature importance data saved to: {excel_path}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution pipeline"""

    # Step 1: Load feature recommendations
    feature_source_map = load_feature_recommendations(BEST_FEATURES_FILE)

    # Step 2: Build mixed-source dataset
    df = build_mixed_source_dataset(feature_source_map, FILE_MAPPING)

    # ========================================================================
    # SAVE FINAL COMBINED DATASET - ADD THIS CODE HERE
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING FINAL COMBINED DATASET")
    print("=" * 80)

    # Save the complete dataset (before preprocessing)
    combined_dataset_file = os.path.join(OUTPUT_DIR, 'final_combined_dataset.xlsx')
    df.to_excel(combined_dataset_file, index=False)
    print(f"✓ Final combined dataset saved to: {combined_dataset_file}")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

    # Also save as CSV for easier access
    combined_dataset_csv = os.path.join(OUTPUT_DIR, 'final_combined_dataset.csv')
    df.to_csv(combined_dataset_csv, index=False)
    print(f"✓ Final combined dataset (CSV) saved to: {combined_dataset_csv}")

    # Save a summary of data sources for each feature
    feature_source_summary = pd.DataFrame([
        {'Feature': feature, 'Source': source}
        for feature, source in feature_source_map.items()
    ])
    source_summary_file = os.path.join(OUTPUT_DIR, 'feature_source_mapping.xlsx')
    feature_source_summary.to_excel(source_summary_file, index=False)
    print(f"✓ Feature-source mapping saved to: {source_summary_file}")
    print("=" * 80)
    # ========================================================================

    # Step 3: Preprocess data
    X, y = preprocess_data(df)

    # Step 4: Split and scale
    X_train, X_test, y_train, y_test, scaler = split_and_scale_data(X, y)

    # ========================================================================
    # SAVE PREPROCESSED DATASETS - ADD THIS CODE HERE (OPTIONAL)
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING PREPROCESSED DATASETS")
    print("=" * 80)

    # Save train and test sets (before scaling)
    train_df = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
    test_df = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

    train_file = os.path.join(OUTPUT_DIR, 'train_set.xlsx')
    test_file = os.path.join(OUTPUT_DIR, 'test_set.xlsx')

    train_df.to_excel(train_file, index=False)
    test_df.to_excel(test_file, index=False)

    print(f"✓ Training set saved to: {train_file}")
    print(f"  Shape: {train_df.shape}")
    print(f"✓ Test set saved to: {test_file}")
    print(f"  Shape: {test_df.shape}")
    print("=" * 80)
    # ========================================================================

    # Step 5: Benchmark all models
    results_df, trained_models, predictions, confusion_matrices = benchmark_all_models(
        X_train, X_test, y_train, y_test
    )

    # Step 6: Visualizations for all models
    print("\n" + "=" * 80)
    print("GENERATING VISUALIZATIONS")
    print("=" * 80)

    plot_model_comparison(
        results_df,
        os.path.join(OUTPUT_DIR, 'model_comparison.png')
    )

    plot_confusion_matrices(
        confusion_matrices,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_all.png')
    )

    plot_roc_curves(
        predictions,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_all.png')
    )

    # Step 7: Select top models for tuning
    top_models = results_df.head(TOP_N_MODELS_TO_TUNE)['Model'].tolist()
    print(f"\n" + "=" * 80)
    print(f"TOP {TOP_N_MODELS_TO_TUNE} MODELS FOR HYPERPARAMETER TUNING:")
    for i, model_name in enumerate(top_models, 1):
        print(f"  {i}. {model_name}")
    print("=" * 80)

    # Step 8: Hyperparameter tuning
    tuned_models = tune_top_models(top_models, trained_models, X_train, y_train)

    # Step 9: Final evaluation
    final_df, best_model, best_model_name, final_preds, final_cms = final_evaluation(
        tuned_models, X_train, X_test, y_train, y_test
    )

    # Step 10: Final visualizations
    plot_confusion_matrices(
        final_cms,
        os.path.join(OUTPUT_DIR, 'confusion_matrices_tuned.png')
    )

    plot_roc_curves(
        final_preds,
        y_test,
        os.path.join(OUTPUT_DIR, 'roc_curves_tuned.png')
    )

    # Step 11: Feature importance for best model
    plot_feature_importance(
        best_model,
        X.columns.tolist(),
        best_model_name,
        os.path.join(OUTPUT_DIR, f'feature_importance_{best_model_name.replace(" ", "_")}.png')
    )

    # Step 12: Generate classification report for best model
    print("\n" + "=" * 80)
    print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
    print("=" * 80)
    y_pred_best = best_model.predict(X_test)
    print(classification_report(y_test, y_pred_best, target_names=['AI', 'Human']))

    # Save classification report
    report_dict = classification_report(y_test, y_pred_best, target_names=['AI', 'Human'], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_file = os.path.join(OUTPUT_DIR, f'classification_report_{best_model_name.replace(" ", "_")}.xlsx')
    report_df.to_excel(report_file)
    print(f"\n✓ Classification report saved to: {report_file}")

    # Save scaler
    scaler_file = os.path.join(OUTPUT_DIR, 'scaler.pkl')
    joblib.dump(scaler, scaler_file)
    print(f"✓ Scaler saved to: {scaler_file}")

    print("\n" + "=" * 80)
    print("🎉 ML BENCHMARKING PIPELINE COMPLETE!")
    print("=" * 80)
    print(f"📁 All results saved to: {OUTPUT_DIR}/")
    print(f"🏆 Best Model: {best_model_name}")
    print(f"📊 F1-Score: {final_df.iloc[0]['F1-Score']:.4f}")
    print(f"📊 Accuracy: {final_df.iloc[0]['Accuracy']:.4f}")
    print("=" * 80)

if __name__ == "__main__":
    main()

AI vs HUMAN TEXT CLASSIFICATION - ML BENCHMARKING SYSTEM
Class Labels: AI = 0, Human = 1
Test Size: 0.2, CV Folds: 5, Random Seed: 42

Loading feature recommendations from: best_features_summary.xlsx
Total features in file: 21
Highly recommended features: 12

Highly Recommended Features and Their Sources:
  - algebraic_connectivity: adv_bnlp
  - avg_shortest_path_length: adv_bnlp
  - avg_closeness_centrality: adv_bnlp
  - transitivity: adv_bnlp
  - avg_betweenness_centrality: adv_bnlp
  - diameter: simple_bnlemma
  - radius: simple_bnlemma
  - top_eigenvalue: adv_bnlp
  - avg_clustering: adv_bnlp
  - wiener_index: simple_bnlemma
  - avg_degree_centrality: simple_bnlemma
  - density: simple_bnlemma

BUILDING MIXED-SOURCE DATASET

Processing feature: algebraic_connectivity from source: adv_bnlp
  ✓ Successfully loaded from both AI and Human files

Processing feature: avg_shortest_path_length from source: adv_bnlp
  ✓ Successfully loaded from both AI and Human files

Processing feature: a